In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Visualize the test data
import scipy.io as io

# ---------------------------------------------- Display the Image ----------------------------------------------------
test_voxels = np.load("Full_Testdata/cup/30/test/cup_000000315_1.npy")
#test_voxels = np.pad(test_voxels,(1,1), 'constant', constant_values=(0,0)) # aus 30x30x30 wird 32x32x32

#test_voxels = test_voxels.astype(int)

print ("Shape:", test_voxels.shape)

print ("Sample", test_voxels)

fig = plt.figure()
bx = fig.gca(projection='3d')
bx.set_aspect('equal')
bx.voxels(test_voxels, edgecolor='red')
plt.show(test_voxels)

In [ ]:
def conv3d(x,W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')

def maxpool3d(x):
    #                          size of window         movement of window   in 3D kommt einfach eine Dimension dazu
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

def convolutional_neural_network(x):

    weights = {'W_conv1': tf.Variable(tf.random_normal([3, 3, 3, 1, 32])), #5x5x5 Convolution, 1 Input und 32 Features/Outputs
               'W_conv2': tf.Variable(tf.random_normal([3, 3, 3, 32, 64])),# 5x5x5 Convolution, 32 Inputs(von conv1) und 64 Outputs
               'W_fc': tf.Variable(tf.random_normal([32768, 1024])),
               'out': tf.Variable(tf.random_normal([1024, n_classes]))}
    '''
    Berechnung aus MNIST Pro Tut: IMG= 32*32*32 - nach maxpool3d mit ksize= 2*2*2 = 16*16*16
                                  IMG= 16*16*16 - nach dem zweiten Durchlauf: 8*8*8
                                  Je nachdem wie groß die Anzahl der Features ist berechnet sich der Wert : 8*8*8*64 = 
    '''
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
              'b_conv2': tf.Variable(tf.random_normal([64])),
              'b_fc': tf.Variable(tf.random_normal([1024])),
              'out': tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_SIZE_X, IMG_SIZE_Y, IMG_SIZE_Z, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)

    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2, [-1, 32768])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate) #mimicing dead neurons, 80% of neurons will be kept (keep_rate = 0.8) helps fighting local maxima

    output = tf.matmul(fc, weights['out']) + biases['out']

    return output

NN_name = "3DCNN_Model"
file_path = './' + NN_name + '/'

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, file_path + 'model.checkpoint')

session.run(y_pred, feed_dict={x: test_voxels})
